In [73]:
# Import relevant libraries
from splinter import Browser
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import os
import pandas as pd
import time

In [81]:
seasons = list(range(2019,2025))
seasons

[2019, 2020, 2021, 2022, 2023, 2024]

In [75]:
# Set up the browser
executable_path = {'executable_path':"C:\Program Files (x86)\msedgedriver.exe"}
browser = Browser('edge', **executable_path)

In [76]:
def read_season_info(soup):
    nav = soup.find('div', id='bottom_nav_container')
    hrefs = [a["href"] for a in nav.find_all("a")]
    season = hrefs[1].split()[0].split('_')[0].split('/')[-1]
    return season

In [77]:
# Check to see if the page has loaded
def page_loaded(browser):
    return browser.is_element_present_by_id('line_score', wait_time=5)

games = []

for season in seasons:
    # Visit the website for scraping
    url = f"https://www.basketball-reference.com/leagues/NBA_{season}_games.html"
    browser.visit(url)
    time.sleep(2)
    print(f'Gather data for {season}')
    
    # Check if the page has loaded
    if not page_loaded(browser):
        # If not, reload the page and wait
        browser.reload()
        time.sleep(3)

    # Create a BeautifulSoup object
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    table = soup.find('div', id='content')

    if table is not None:
        month_filter = table.find('div', class_='filter')
        links = month_filter.find_all("a")
        hrefs = [link.get("href") for link in links]
        standing_pages = [f"https://www.basketball-reference.com{link}" for link in hrefs[:-1]]
        
        for page in standing_pages:
            browser.visit(page)
            time.sleep(2) 
            
            # Create a BeautifulSoup object
            html = browser.html
            soup_page = BeautifulSoup(html, 'html.parser')

            links = soup_page.find_all("a")
            hrefs = [link.get("href") for link in links]
            
             # Filer out 2024 links
            box_scores = [link for link in hrefs if link and "boxscore" in link and ".html" in link and '2024' not in link]
            box_scores = [f"https://www.basketball-reference.com{score}" for score in box_scores]

            base_cols = None
            for box_score in box_scores:
                browser.visit(box_score)
                time.sleep(2)
                
                html=browser.html
                soup_box = BeautifulSoup(html, 'html.parser')
                
                score_table = soup_box.find('table', id='line_score')
                
                # Cheack to see if score_table is None
                if score_table is None:
                    browser.reload()
                    time.sleep(2)
                    
                    # Re-create the BeautifulSoup object
                    html=browser.html
                    soup_box = BeautifulSoup(html, 'html.parser')
                    score_table = soup_box.find('table', id='line_score')
                
                tbody = score_table.find('tbody')
                rows = tbody.find_all('tr')

                line_score = []
                for row in rows:
                    # Get team name
                    team = row.find('th', class_='center').text
                    columns = row.find_all('td')

                    # Create line score dictionary
                    if(columns !=[]):
                        total = columns[4].text

                    # Create dictionary for dataframe later
                    line_score_dict = { "team": team,
                                        "total": total
                    }

                    # Add dictionary to array
                    line_score.append(line_score_dict)

                # Create Data frame
                score_df = pd.DataFrame(line_score)
                teams = [score["team"] for score in line_score]

                summaries = []
                for team in teams:
                    # Convert html table into pandas dataframe
                    basic = pd.read_html(str(soup_box), attrs={"id": f"box-{team}-game-basic"}, index_col=0)[0]
                    advanced = pd.read_html(str(soup_box), attrs={"id": f"box-{team}-game-advanced"}, index_col=0)[0]

                    # Convert all columns with number to numeric for dataframes
                    basic = basic.apply(pd.to_numeric, errors="coerce")
                    advanced = advanced.apply(pd.to_numeric, errors="coerce")

                    # Remove the row that contains heards within the dataframe
                    advanced = advanced.drop('Reserves')
                    basic = basic.drop('Reserves')

                    # Get headers for basic and advanced stats
                    advanced_columns = []
                    basic_columns = []
                    for i in range(len(advanced.columns)):
                        advanced_columns.append(advanced.columns[i][1])

                    for i in range(len(basic.columns)):
                        basic_columns.append(basic.columns[i][1])

                    advanced.columns = advanced_columns
                    basic.columns = basic_columns

                    # Totals and Maxes
                    totals = pd.concat([basic.iloc[-1, :], advanced.iloc[-1, :]])
                    totals.index = totals.index.str.lower()

                    maxes = pd.concat([basic.iloc[:-1, :].max(), advanced.iloc[:-1, :].max()])
                    maxes.index = maxes.index.str.lower() + "_max"

                    summary = pd.concat([totals, maxes])

                    # Create common columns that are found for all teams
                    if base_cols is None:
                        base_cols = list(summary.index.drop_duplicates(keep="first"))
                        base_cols = [b for b in base_cols if "bpm" not in b]

                    summary = summary[base_cols]
                    summaries.append(summary)

                summary = pd.concat(summaries, axis=1).T
                game = pd.concat([summary, score_df], axis=1)
                game["home"] = [0, 1]

                game_opp = game.iloc[::-1].reset_index()
                game_opp.columns += "_opp"

                # Merge both home and away team data together
                full_game = pd.concat([game, game_opp], axis=1)

                # Add the season the game was played in
                full_game["season"] = read_season_info(soup_box)

                # Add date to dataframe
                full_game["date"] = box_score.split('/')[-1][:8]
                full_game["date"] = pd.to_datetime(full_game["date"], format="%Y%m%d")

                # Specify who won the game
                full_game["won"] = full_game["total"] > full_game["total_opp"]

                games.append(full_game)

                if len(games) % 100 == 0:
                    print(f"{len(games)} / {len(box_scores)}")
browser.quit()

100 / 225
200 / 225
300 / 225
400 / 209
500 / 209
600 / 231
700 / 231
800 / 163
900 / 163
1000 / 229
1100 / 229
1200 / 129
1300 / 38
1400 / 102
1500 / 222
1600 / 222
1700 / 222
1800 / 222
1900 / 222
2000 / 222
2100 / 163
2200 / 163
2300 / 229
2400 / 229
2500 / 121
2600 / 34
2700 / 219
2800 / 219
2900 / 219
3000 / 208
3100 / 208


In [78]:
games_df = pd.concat(games, ignore_index=True)

In [79]:
games_df.to_csv("nba_games.csv")